In [1]:
%%time

import glob

import geopandas as gpd
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
from libpysal.graph import read_parquet
from sklearn.preprocessing import PowerTransformer, RobustScaler, StandardScaler

from fast_hdbscan.numba_kdtree import kdtree_to_numba
from fast_hdbscan.numba_kdtree import parallel_tree_query
from sklearn.neighbors import KDTree

CPU times: user 11.1 s, sys: 400 ms, total: 11.5 s
Wall time: 9.14 s


In [2]:
region_id = 69300

tessellations_dir = '/data/uscuni-ulce/processed_data/tessellations/'
chars_dir = "/data/uscuni-ulce/processed_data/chars/"
graph_dir = "/data/uscuni-ulce/processed_data/neigh_graphs/"

In [3]:
graph = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")

In [4]:
graph.cardinalities.describe()

count    304554.000000
mean          6.751085
std           2.060782
min           1.000000
25%           6.000000
50%           7.000000
75%           8.000000
max          82.000000
Name: cardinalities, dtype: float64

In [5]:
from core.cluster_validation import print_distance, generate_neigbhourhood_groups, generate_detailed_clusters

In [6]:
tessellation = gpd.read_parquet(
        tessellations_dir + f"tessellation_{region_id}.parquet"
)

In [7]:
X_train = pd.read_parquet(chars_dir + f'primary_chars_{region_id}.parquet')



X_train = X_train[X_train.index >= 0]



spatial_lag = 1


# lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/context_chars_{region_id}_lag_{spatial_lag}.parquet')

lag = pd.read_parquet(f'/data/uscuni-ulce/processed_data/context_data/unprocessed_context_chars_{region_id}_lag_{spatial_lag}.parquet')



X_train = X_train.join(lag[[c for c in lag.columns if '_median' in c]], how='inner')

# X_train = X_train.join(lag, how='inner')


In [8]:
# for c in X_train.columns:
#     X_train[c] = X_train[c].clip(*np.percentile(X_train[c], [5, 95]))

In [9]:
to_drop = ['stcSAl',
 'ltkOri',
 'stbOri',
 'stcOri',
 'stbCeA']


all_drop = []
for c in to_drop:
    all_drop += X_train.columns[X_train.columns.str.contains(c)].tolist()


X_train = X_train.drop(all_drop, axis=1)

In [10]:
vals = StandardScaler().fit_transform(X_train)
X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)

vals = np.nan_to_num(X_train)
X_train = pd.DataFrame(vals, columns=X_train.columns, index=X_train.index)


# X_train = X_train.clip(-10, 10)

In [11]:
# t1 = X_train[[c for c in X_train.columns if '_' not in c]]
# t2 = X_train[[c for c in X_train.columns if '_median' in c]]

# X_train = t1.join(t2)
# X_train.shape

In [12]:
stats = X_train.describe()
stats.columns[stats.loc['std'] == 0]

Index([], dtype='object')

In [13]:
X_train = X_train.drop(stats.columns[stats.loc['std'] == 0], axis=1)

In [14]:
X_train.shape

(299064, 116)

In [15]:
tess_groups = generate_detailed_clusters(tessellation,
                                         include_random_sample=False)
tess_groups = tess_groups[tess_groups.index.isin(X_train.index)]
tess_groups_ilocs = (
    pd.Series(np.arange(len(X_train)), index=X_train.index)
    .loc[tess_groups.index]
    .values
)

def check_score(data, example_clusters):
    groups = example_clusters[example_clusters.index.isin(data.index)]
    groups_ilocs = (
        pd.Series(np.arange(len(data)), index=data.index).loc[groups.index].values
    )
    return davies_bouldin_score(data.iloc[groups_ilocs], groups.values)

In [16]:
# tessellation.loc[tess_groups.index].explore()

In [17]:
from core.cluster_validation import print_distance
print_distance( pd.DataFrame(X_train.loc[tess_groups.index]).groupby(tess_groups.values).mean(), metric='euclidean')

,commie blocks vn,fancy commie blocks,holyne,housing blocks,housing houses,josefov,karlin IT offices,karlin old,karlin river offices,karlin square,mala strana,malesice,prague castle,row houses1,row houses2,smickov,stare mesto,vinohrady blocks,vinohrady squares,vinohrady villas
commie blocks vn,0.000000,6.220750,8.521257,6.573341,8.182632,11.086921,7.515662,13.027184,13.252763,8.594146,15.041953,9.597918,12.185749,11.338858,13.236321,10.348689,15.452198,7.048055,11.874175,8.444788
fancy commie blocks,6.220750,0.000000,9.686341,5.331128,9.534352,13.353992,6.862914,13.341675,10.948498,10.582111,15.518968,7.827221,11.053686,12.395313,13.973408,11.683256,16.529093,8.141783,12.782121,9.294975
holyne,8.521257,9.686341,0.000000,8.460887,6.160017,11.891323,9.258167,14.101841,14.513258,10.205214,14.945877,9.222568,11.786558,11.953267,13.087703,11.275792,15.653695,6.558599,12.685826,6.850911
housing blocks,6.573341,5.331128,8.460887,0.000000,7.699806,11.224305,6.280551,12.551617,10.342794,9.475236,14.481901,7.869497,9.808158,12.595769,14.199573,10.100979,14.754254,6.334880,11.939035,7.923012
housing houses,8.182632,9.534352,6.160017,7.699806,0.000000,9.707536,7.488744,12.565810,14.883248,7.774523,15.766394,10.102736,12.879311,10.615952,12.095600,8.751993,15.140807,6.296328,9.250980,3.346710
josefov,11.086921,13.353992,11.891323,11.224305,9.707536,0.000000,11.912355,9.685122,17.803097,5.709831,11.980336,14.312127,11.734107,13.852085,15.451410,4.939865,9.073252,10.992656,7.839815,10.073625
karlin IT offices,7.515662,6.862914,9.258167,6.280551,7.488744,11.912355,0.000000,12.007769,11.480888,9.131926,15.782902,6.447580,12.355578,10.912745,12.443861,9.701321,15.897843,5.655634,10.785907,6.965297
karlin old,13.027184,13.341675,14.101841,12.551617,12.565810,9.685122,12.007769,0.000000,18.237355,7.355494,9.860030,13.327562,9.931066,12.133446,13.498452,6.407114,8.384132,12.275076,7.943733,11.964370
karlin river offices,13.252763,10.948498,14.513258,10.342794,14.883248,17.803097,11.480888,18.237355,0.000000,16.327845,19.060829,10.837472,15.526523,18.022962,19.203957,16.524951,20.238422,11.108278,17.996696,14.528481
karlin square,8.594146,10.582111,10.205214,9.475236,7.774523,5.709831,9.131926,7.355494,16.327845,0.000000,12.499504,11.551455,10.824826,10.427053,12.288572,3.132282,10.692928,8.868458,5.172792,7.817881


In [18]:
from scipy.spatial.distance import pdist
for i, g in X_train.loc[tess_groups.index].groupby(tess_groups.values):
    print(i, np.mean(pdist(g)))

commie blocks vn 9.859530824217407
fancy commie blocks 11.256658082308165
holyne 11.008416845479665
housing blocks 12.871346212789568
housing houses 8.231457431861335
josefov 10.41005338725719
karlin IT offices 11.06116174308631
karlin old 12.550588776765602
karlin river offices 17.113722335513565
karlin square 11.966327606167653
mala strana 21.02230146430161
malesice 17.116622074069713
prague castle 23.712959805930936
row houses1 9.346221622428152
row houses2 7.598653531598445
smickov 11.60581611159095
stare mesto 16.662740037251982
vinohrady blocks 9.361792362079568
vinohrady squares 7.81206628288167
vinohrady villas 7.942507273342982


In [19]:
# tessellation.loc[tess_groups.index].explore(column=tess_groups.values, categorical=True)

In [20]:
mean_clusters = pd.DataFrame(X_train.loc[tess_groups.index]).groupby(tess_groups.values).mean()

In [21]:
(mean_clusters.loc['josefov'] - mean_clusters.loc['vinohrady squares']).abs().sort_values(ascending=False)

linP4W_median    2.861208
linP4W           2.847238
linP3W_median    2.749055
linP3W           2.688568
ssbSqu_median    1.944645
                   ...   
ldkAre_median    0.004477
ldkAre           0.003503
ltkWNB_median    0.001734
sdcLAL_median    0.000985
sdbCoA_median    0.000005
Length: 116, dtype: float64

In [22]:
(mean_clusters.loc['josefov'] - mean_clusters.loc['stare mesto']).abs().sort_values(ascending=False)

libNCo_median    4.900441
libNCo           4.866751
ssbERI_median    1.881506
ldbPWL_median    1.836020
ldbPWL           1.788455
                   ...   
sdcLAL           0.011426
sdbAre           0.008415
midRea_median    0.006295
midAre_median    0.004590
ssbCor           0.000406
Length: 116, dtype: float64

In [23]:
from core.utils import used_keys
used_keys['ldbPWL']

'perimeter wall length of adjacent buildings'

In [24]:
# training_data = X_train.loc[tess_groups.index]
# tess_groups_ilocs = (
#     pd.Series(np.arange(len(training_data)), index=training_data.index)
#     .loc[tess_groups.index]
#     .values
# )
# training_data.shape

In [25]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import fcluster
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import davies_bouldin_score
from core.cluster_validation import get_linkage_matrix

In [26]:
q1 = read_parquet(graph_dir + f"tessellation_graph_{region_id}_knn1.parquet")

# clustering_graph = q1.higher_order(k=3, lower_order=True, diagonal=True).subgraph(X_train.index.values)

clustering_graph = q1.copy()

In [27]:
graph_labels = q1.subgraph(X_train.index.values).component_labels
graph_labels.value_counts()

component labels
776    60555
452    25177
754    17243
99     15583
726    13382
       ...  
247        1
727        1
249        1
250        1
161        1
Name: count, Length: 919, dtype: int64

In [28]:
clustering_graph = clustering_graph.subgraph(graph_labels[graph_labels == 776].index.values)

In [29]:
core_ids = clustering_graph.unique_ids
clustering_graph = clustering_graph.transform('B').sparse

In [30]:
training_data = X_train[X_train.index.isin(core_ids)]

In [31]:


# training_data = training_data[[c for c in training_data.columns if '_' not in c]]


In [32]:
# t1 = training_data[[c for c in training_data.columns if '_' not in c]]
# t2 = training_data[[c for c in training_data.columns if '_median' in c]]

# training_data = t1.join(t2)

In [33]:
# training_data = X_train[X_train.index >=0]

In [34]:
training_data.shape

(60555, 116)

In [35]:
# training_data = training_data[[c for c in training_data.columns if '_' not in c]]
# training_data.shape

In [36]:
%%time
clusterer = AgglomerativeClustering(linkage='ward',
                                    connectivity=clustering_graph, 
                                    # connectivity=q1.subgraph(X_train.index.values).transform('B').sparse, 
                                    compute_full_tree=True,
                                    compute_distances=True)
model = clusterer.fit(training_data)

CPU times: user 1.62 s, sys: 48.5 ms, total: 1.67 s
Wall time: 1.67 s


In [37]:
linkage_matrix = get_linkage_matrix(model)

In [38]:
# fix, ax = plt.subplots(figsize=(40,40))
# # Plot the corresponding dendrogram
# _ = dendrogram(linkage_matrix, truncate_mode="level", p=5, ax=ax)

In [39]:
from sklearn.metrics import calinski_harabasz_score

In [40]:
tess_groups = generate_detailed_clusters(tessellation,
                                         include_random_sample=False)
tess_groups = tess_groups[tess_groups.index.isin(training_data.index)]
tess_groups_ilocs = (
    pd.Series(np.arange(len(training_data)), index=training_data.index)
    .loc[tess_groups.index]
    .values
)

In [41]:
for t in range(25, 1000, 25):
    r = fcluster(linkage_matrix, t=t, criterion='distance')
    # r = pd.Series(r, index=X_train.index)
    # # ssplits = graph.describe(r, statistics=['nunique'])['nunique']
    print(t, ' - ', 
          adjusted_rand_score(tess_groups.values, r[tess_groups_ilocs]),
          # (ssplits > 1).sum() / ssplits.shape[0],
          davies_bouldin_score(training_data, r),
          calinski_harabasz_score(training_data, r)
         )

25  -  0.15104935483658732 2.1083772268680128 78.31693731681537
50  -  0.4143582211521189 2.798058613335092 156.82543827998614
75  -  0.5450933869258779 3.3500751010017327 241.94116531965096
100  -  0.6131980548506211 3.731557998011874 320.0293358301426
125  -  0.6332579139388193 3.9116026885299395 422.09901727247524
150  -  0.6599409183611245 4.009266317518327 513.1602895454402
175  -  0.6737997342413939 4.252400167683714 602.7488082963455
200  -  0.49952851177796714 4.504741479000952 721.0687272148965
225  -  0.387744275437628 4.571672459744415 821.5406685098376
250  -  0.387744275437628 3.9598080293098055 1032.6472391132777
275  -  0.387744275437628 3.9598080293098055 1032.6472391132777
300  -  0.3664178308059327 3.653042281476848 1177.3686685426503
325  -  0.35097310926627273 3.714098317249412 1401.5357148349235
350  -  0.35097310926627273 4.200674862795946 1621.8240858088964
375  -  0.3105119562515747 4.509285083977414 1721.4532796455276
400  -  0.3105119562515747 5.0627116736156 

ValueError: Number of labels is 1. Valid values are 2 to n_samples - 1 (inclusive)

In [ ]:
# for t in range(5, 25, 1):
#     r = fcluster(linkage_matrix, t=t, criterion='distance')
#     # r = pd.Series(r, index=X_train.index)
#     # # ssplits = graph.describe(r, statistics=['nunique'])['nunique']
#     print(t, ' - ', 
#           adjusted_rand_score(tess_groups.values, r[tess_groups_ilocs]),
#           # (ssplits > 1).sum() / ssplits.shape[0],
#           davies_bouldin_score(training_data, r),
#           calinski_harabasz_score(training_data, r)
#          )

In [42]:
plotting = tessellation.loc[training_data.index].reset_index()

In [43]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 1.15 s, sys: 84 ms, total: 1.23 s
Wall time: 1.23 s


In [44]:
from sidecar import Sidecar
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [45]:
from core.cluster_validation import get_color

In [50]:
clusters = fcluster(linkage_matrix, t=100, criterion='distance')

In [51]:
np.unique(clusters, return_counts=True)

(array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178

In [56]:
layer.get_fill_color = get_color(clusters)

In [49]:
new_data = training_data.groupby(clusters).mean()

In [55]:
# try hdbscan extraction
from fast_hdbscan.boruvka import parallel_boruvka
from fast_hdbscan.cluster_trees import (
    cluster_tree_from_condensed_tree,
    condense_tree,
    extract_eom_clusters,
    get_cluster_label_vector,
    mst_to_linkage_tree,
)
from fast_hdbscan.numba_kdtree import kdtree_to_numba
from sklearn.neighbors import KDTree

condensed_tree = condense_tree(linkage_matrix, 
                               min_cluster_size=100)
cluster_tree = cluster_tree_from_condensed_tree(condensed_tree)
selected_clusters = extract_eom_clusters(
    condensed_tree, cluster_tree, allow_single_cluster=False
)
clusters = get_cluster_label_vector(condensed_tree, selected_clusters, 0)

In [44]:
for min_cluster_size in range(25, 500, 25):


    condensed_tree = condense_tree(linkage_matrix, 
                                   min_cluster_size=min_cluster_size)
    cluster_tree = cluster_tree_from_condensed_tree(condensed_tree)
    selected_clusters = extract_eom_clusters(
        condensed_tree, cluster_tree, allow_single_cluster=False
    )
    r = get_cluster_label_vector(condensed_tree, selected_clusters, 0)

    print(min_cluster_size, ' - ', 
              adjusted_rand_score(tess_groups.values, r[tess_groups_ilocs]),
              # (ssplits > 1).sum() / ssplits.shape[0],
              davies_bouldin_score(training_data, r),
              calinski_harabasz_score(training_data, r)
             )

25  -  0.15922510083983504 2.9410872146217826 65.61446939389633
50  -  0.3329173402061008 3.4168411906459957 104.63822842799746
75  -  0.444007858819182 3.7759213479636475 131.18048714215274
100  -  0.49908390227250354 4.0621685530629135 158.19198587764362
125  -  0.5027023971134222 4.230026431915797 177.10353099824832
150  -  0.5665983244675065 4.386828664517716 200.079573352324
175  -  0.5645679927227408 4.559119544402977 210.90782701036667
200  -  0.5645877854397119 4.765522153234293 225.11923197494087
225  -  0.5171287589282915 4.8303139253265535 235.4354948742005
250  -  0.6436070855185897 5.139003495389489 277.8785211989204
275  -  0.6426057911962102 5.026906540988767 310.8428037984367
300  -  0.6234295846535862 5.053793486604022 352.0749298833545
325  -  0.627233774092026 5.127235128938351 373.489787957305
350  -  0.6348698949200634 5.248311920173215 397.64845381956036
375  -  0.6538084702670228 5.402044497528021 415.3763009900913
400  -  0.6538084702670228 5.393918569502529 424

In [ ]:
for t in range(25, 1000, 25):
    r = fcluster(linkage_matrix, t=t, criterion='distance')
    # r = pd.Series(r, index=X_train.index)
    # # ssplits = graph.describe(r, statistics=['nunique'])['nunique']
    

In [157]:
from core.cluster_validation import get_feature_importance
from core.utils import used_keys

In [69]:
clusters_subset = [11597, 11615, 17742]
clusters_subset = np.where(np.isin(clusters, clusters[clusters_subset]))

In [70]:
imps = get_feature_importance(training_data.iloc[clusters_subset], clusters[clusters_subset])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [72]:
imps.loc[:10, [c for c in imps.columns if '_vals' not in c]]

,cluster_48,cluster_66,cluster_69
0,linWID_higher,lcdMes_higher,ssbCCD_lower
1,ldsMSL_lower,linPDE,ltkOri_lower
2,lddNDe,linP3W_higher,ssbSqu_lower
3,ltkOri_lower,mtbAli_median,lcdMes_median
4,ldsMSL,lskCWA_lower,linWID_higher
5,ldsMSL_median,lskERI_median,sicCAR_lower
6,midRea_median,ltkWNB_median,lskCWA_lower
7,ldkAre_higher,linWID_lower,stcOri_lower
8,lskCCo_lower,mtbNDi_higher,ssbSqu_median
9,ssbCCD_lower,linPDE_lower,sdcLAL_median


In [77]:
used_keys['ltkOri']

'orientation of enclosure'

In [112]:
imps = get_feature_importance(training_data, clusters)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done   4 out of  10 | elapsed:    0.2s remaining:    0.3s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 20 co

In [119]:
imps.loc[:10, [c for c in imps.columns if '_vals' not in c]]

,cluster_1,cluster_2,cluster_3,cluster_4,cluster_5,cluster_6,cluster_7
0,ldkPer,lcdMes,libNCo,lskCWA,ldsAre,ltkWNB,likWBB
1,ltkWNB,likWBB,likWBB,likWBB,midAre,ldkPer,lcdMes
2,lskCCo,ltkWNB,sdsSPO,ltkWNB,ltkOri,lskCCo,ltkWNB
3,ltkOri,lskERI,ldbPWL,lskCCo,linP4W,lskCWA,linWID
4,lskCWA,lskCWA,ltcBuA,ldkAre,sdsSPO,ltkOri,ltkOri
5,ldkAre,ldkAre,sicCAR,lskERI,ldsMSL,likWBB,lskCWA
6,likWBB,linP4W,mtbSWR,ldsAre,lddNDe,ldkAre,ldkAre
7,lcdMes,ldkPer,ssbSqu,lddNDe,lcdMes,lskERI,lddNDe
8,lskERI,ltkOri,lskERI,midRea,ldkPer,linP3W,linP3W
9,linP4W,linPDE,sdbPer,linPDE,likWBB,ldsAre,lcnClo


In [122]:
used_keys['lcdMes']

'local meshedness of street network'

In [ ]:
imps[[c for c in imps.columns if '_vals' in c]].cumsum(axis=1)

In [96]:
josefov_joins = []

josefov_joins.append(np.isin(linkage_matrix[:, 0], 
                             tess_groups_ilocs[tess_groups == 'josefov']))
josefov_joins.append(np.isin(linkage_matrix[:, 1], 
                             tess_groups_ilocs[tess_groups == 'josefov']))


In [97]:
indxs = linkage_matrix[josefov_joins[0] | josefov_joins[1]]
indxs = np.union1d(indxs[:, 0], indxs[:, 1])
indxs = indxs[indxs <= X_train.shape[0]]

In [150]:
indxs = linkage_matrix[linkage_matrix[:, 2] <= 2]
indxs = np.union1d(indxs[:, 0], indxs[:, 1])
indxs = indxs[indxs < X_train.shape[0]]
indxs.shape

(64349,)

In [151]:
plotting = tessellation.loc[X_train.iloc[indxs].index]

In [42]:
cluster_means = training_data.groupby(clusters).mean()

In [68]:
c1 = 6
c2 = 10

(cluster_means.loc[c1] - cluster_means.loc[c2]).sort_values(ascending=False)

libNCo           6.284668
libNCo_median    5.846288
libNCo_higher    4.233946
linPDE_higher    3.014914
linPDE           2.122206
                   ...   
lcnClo          -1.414220
linWID_lower    -1.422183
linP3W          -1.489277
lcnClo_lower    -1.702326
linP3W_lower    -2.339139
Length: 248, dtype: float64

In [72]:
from core.utils import used_keys
used_keys['libNCo']

'number of courtyards within adjacent buildings'

In [169]:
bgraph = read_parquet(graph_dir + f"building_graph_{region_id}_knn1.parquet")

In [170]:
buildings_dir = '/data/uscuni-ulce/processed_data/buildings/'

buildings = gpd.read_parquet(
        buildings_dir + f"buildings_{region_id}.parquet"

)

In [178]:
buildings

,index,id,geometry
0,0,v0.1-CZE.12.2_1-35164,"POLYGON ((4614847.626 2975218.938, 4614848.235..."
1,1,v0.1-CZE.12.2_1-35123,"POLYGON ((4615276.357 2976034.184, 4615282.866..."
2,2,v0.1-CZE.12.2_1-35159,"POLYGON ((4615315.503 2975986.2, 4615322.056 2..."
3,3,v0.1-CZE.12.2_1-35166,"POLYGON ((4615222.339 2976016.91, 4615224.582 ..."
4,4,v0.1-CZE.12.2_1-35228,"POLYGON ((4615300.348 2975924.258, 4615301.6 2..."
...,...,...,...
299059,299060,v0.1-CZE.13.3_1-13696,"POLYGON ((4618611.169 3033535.197, 4618623.01 ..."
299060,299061,v0.1-CZE.13.3_1-13674,"POLYGON ((4618611.989 3033568.153, 4618617.119..."
299061,299062,v0.1-CZE.13.3_1-13591,"POLYGON ((4618614.831 3033550.704, 4618628.289..."
299062,299063,v0.1-CZE.13.3_1-13328,"POLYGON ((4618625.628 3033512.926, 4618625.634..."


In [181]:
buildings = buildings.join(X_train, how='inner').drop(['index', 'id'], axis=1)

In [185]:
r = buildings.dissolve(bgraph.component_labels, aggfunc='mean')

In [189]:
plotting = r

In [190]:
%%time
import lonboard
# plotting = tessellation[tessellation.index.isin(X_train.index)].copy()
layer = lonboard.PolygonLayer.from_geopandas(plotting, opacity=.08)

/home/krasen/miniconda3/envs/urban_taxonomy/lib/python3.12/site-packages/lonboard/_geoarrow/ops/reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


CPU times: user 1.65 s, sys: 178 ms, total: 1.83 s
Wall time: 1.82 s


In [191]:
from sidecar import Sidecar
sc = Sidecar(title='Clusters')
m = lonboard.Map(layer, basemap_style=lonboard.basemap.CartoBasemap.Positron)
with sc:
    display(m)

In [156]:
enclosures = gpd.read_parquet(f"/data/uscuni-ulce/processed_data/enclosures/enclosure_{region_id}.parquet")
encl_counts = tessellation.groupby('enclosure_index').count()
encl_counts.columns = ['tessellation']
enclosures['lieWCe'] = encl_counts['tessellation'] / enclosures.geometry.area

7        0.000325
8        0.000224
11       0.000030
12       0.000122
14       0.000638
           ...   
25157    0.000100
25158    0.000029
25159    0.000101
25160    0.000399
25161    0.000081
Length: 15958, dtype: float64

In [251]:
enclosures['lieWCe'] 

7        0.000325
8        0.000224
11       0.000030
12       0.000122
14       0.000638
           ...   
25157    0.000100
25158    0.000029
25159    0.000101
25160    0.000399
25161    0.000081
Name: lieWCe, Length: 15958, dtype: float64

In [164]:
encl_counts['tessellation']

enclosure_index
7        199
8         52
11         1
12        25
14       962
        ... 
25157      9
25158      1
25159     13
25160     15
25161      3
Name: tessellation, Length: 15935, dtype: int64

In [ ]:
# Measure weighted cells within enclosure
merged = enclosures[['eID', 'ldeAre']].merge(encl_counts[['tessellation']], how='left', on='eID')
enclosures['lieWCe'] = merged['tessellation'] / merged['ldeAre']